In [ ]:
#@title Imports
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torchvision 
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt  

# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

: 

# Part 1: Load/Analyze Data

##What is your dataset? Please describe your dataset and where you got it

My dataset is from https://github.com/bananathrowingmachine/FastPartitionExperimentDocs/tree/main/Previous%20Results/Jul%2012%2C%202025/data_tables which are a set of results formed in mid July from a personal project a wrote in May, just to see if I could make a really fast algorith that solves the psuedo-polynomial time partition problem, at least compared to the solution found on Wikipedia (which I did infact beat quite well). The data essentially has 3 x variables and a y variable, where the x variables are the algorithm type (with there being 4 types, the Wikipedia solution, one change I made, another change I made, and then both changes combined together), the amount of integers in the set, and the approximate sum of the set and then the y variable is the average amount of iterations each method took to solve the same randomly(-ish) generated set. For more information on how the experiment was run as a whole you can look here: https://github.com/bananathrowingmachine/FastPartitionExperiment

For this project the data is known as "Data.xlsx", and will be included with whatever folders this notebook is inside of.

##Load your dataset / Implement your dataloader

In [ ]:
sheetData = pd.ExcelFile('Data.xlsx')

# TODO - if not already done, please make a train/val/test split of your dataset

##Dataset Analyis Statistics

In [ ]:
# TODO - code for collecting statistics from your dataset / plots and analysis

TODO - Provide the analysis and statistics of your data

# Part 2: Literature Review

TODO - Follow instructions in the assignment for your literature review

# Part 3: Implement networks in PyTorch

Note of warning here: Depending on how easily you can allocate GPU resources, you may want to make your network much shallower so that you can train it more easily
Aim to have one gradient update take no more than a few seconds
May also want to reduce the number of training steps if training is too slow

In [ ]:
# TODO - implement your PyTorch Module

In [ ]:
# TODO - implement your training loop

In [ ]:
# TODO - plot train and validation accuracy of your model during training

Hint: look at earlier homeworks / resuse code from those to help you here

# Part 4: Run Hyperparameter Experiments

In [ ]:
# TODO - perform hyperparameter grid searches and plot accuracies

In [ ]:
# TODO - perform other experiments and plot accuracies

In [ ]:
# TODO - calculate the final test accuracy

TODO - Explain your experiments above. See assignment doc for more details.